In [1]:
import os
import ee
import geemap
import math
Map = geemap.Map()
ee.Initialize()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [2]:
import urllib.request

In [3]:
#找到上传的数据，只需要保护区数据
geemap.ee_search()

In [4]:
def generateGrid(xmin, xmax, dx, y1, y2):
    xx = ee.List.sequence(xmin, ee.Number(xmax).subtract(dx), dx)
    cells = xx.map(func_uke).flatten()
    return ee.FeatureCollection(cells)

def func_uke(x):
    x1 = ee.Number(x)
    x2 = ee.Number(x).add(ee.Number(dx))
    coords = ee.List([x1, y1, x2, y2])
    rect = ee.Algorithms.GeometryConstructors.Rectangle(coords)
    return ee.Feature(rect)

In [5]:
def func_feh(feature):
    #自然植被                    
    EvergreenNeedleleafForests5 = feature.getNumber('EvergreenNeedleleafForests5')
    EvergreenBroadleafForests5 = feature.getNumber('EvergreenBroadleafForests5')
    DeciduousNeedleleafForests5 = feature.getNumber('DeciduousNeedleleafForests5')
    DeciduousBroadleafForests5 = feature.getNumber('DeciduousBroadleafForests5')
    MixedForests5 = feature.getNumber('MixedForests5')
    ClosedShrublands5 = feature.getNumber('ClosedShrublands5')
    OpenShrublands5 = feature.getNumber('OpenShrublands5')
    WoodySavannas5 = feature.getNumber('WoodySavannas5')
    Savannas5 = feature.getNumber('Savannas5')
    Grasslands5 = feature.getNumber('Grasslands5')
    PermanentWetlands5 = feature.getNumber('PermanentWetlands5')
    Croplands5 = feature.getNumber('Croplands5')
    UrbanBuiltupLands5 = feature.getNumber('UrbanBuiltupLands5')
    CroplandNaturalVegetationMosaics5 = feature.getNumber('CroplandNaturalVegetationMosaics5')
    PermanentSnowandIce5 = feature.getNumber('PermanentSnowandIce5')
    Barren5 = feature.getNumber('Barren5')
    WaterBodies5 = feature.getNumber('WaterBodies5')
    Unclassified5 = feature.getNumber('Unclassified5')

    Land5 = feature.getNumber('Land5')
    
    #feature的面积
    return feature.set({
        #自然植被
        'EvergreenNeedleleafForests5':EvergreenNeedleleafForests5.divide(Land5),
        'EvergreenBroadleafForests5':EvergreenBroadleafForests5.divide(Land5),
        'DeciduousNeedleleafForests5':DeciduousNeedleleafForests5.divide(Land5),
        'DeciduousBroadleafForests5':DeciduousBroadleafForests5.divide(Land5),
        'MixedForests5':MixedForests5.divide(Land5),
        'ClosedShrublands5':ClosedShrublands5.divide(Land5),
        'OpenShrublands5':OpenShrublands5.divide(Land5),
        'WoodySavannas5':WoodySavannas5.divide(Land5),
        'Savannas5':Savannas5.divide(Land5),
        'Grasslands5':Grasslands5.divide(Land5),
        'PermanentWetlands5':PermanentWetlands5.divide(Land5),
        'Croplands5':Croplands5.divide(Land5),
        'UrbanBuiltupLands5':UrbanBuiltupLands5.divide(Land5),
        'CroplandNaturalVegetationMosaics5':CroplandNaturalVegetationMosaics5.divide(Land5),
        'PermanentSnowandIce5':PermanentSnowandIce5.divide(Land5),
        'Barren5':Barren5.divide(Land5),
        'WaterBodies5':WaterBodies5.divide(Land5),
        'Unclassified5':Unclassified5.divide(Land5),

        'Land5':Land5.divide(Land5)
     })

In [6]:
import requests
from retrying import retry
# 全部报错才会报错，如果其中一次正常，则继续执行
# 两次retry之间等待2秒，重试7次
@retry(stop_max_attempt_number=1000, wait_fixed=2000)
def get_request(url):
    response = urllib.request.urlretrieve(url, filename=out_ndvi_stats)
    return response

In [7]:
dx = 1/120
xmin = -180
xmax = 180
ymin = -65
ymax = 90
yrange = round((ymax - ymin  - 0.0001)/dx)+1
print(yrange)

18601


In [ ]:
#读取土地利用
for year_index in range(0,21):
    year = 2000 + year_index
    year_begin = ee.Date.fromYMD(year,1,1)
    year_end = ee.Date.fromYMD(year,12,31)
    image_lc = ee.ImageCollection('MODIS/006/MCD12Q1').filterDate(year_begin , year_end).select('LC_Type1').mean()
    #合并土地利用
    EvergreenNeedleleafForests = image_lc.eq(1)
    EvergreenBroadleafForests = image_lc.eq(2)
    DeciduousNeedleleafForests = image_lc.eq(3)
    DeciduousBroadleafForests = image_lc.eq(4)
    MixedForests = image_lc.eq(5)
    ClosedShrublands = image_lc.eq(6)
    OpenShrublands = image_lc.eq(7)
    WoodySavannas = image_lc.eq(8)
    Savannas = image_lc.eq(9)
    Grasslands = image_lc.eq(10)
    PermanentWetlands = image_lc.eq(11)
    Croplands = image_lc.eq(12)
    UrbanBuiltupLands = image_lc.eq(13)
    CroplandNaturalVegetationMosaics = image_lc.eq(14)
    PermanentSnowandIce = image_lc.eq(15)
    Barren = image_lc.eq(16)
    WaterBodies = image_lc.eq(17)
    Unclassified = image_lc.eq(255)

    Land = image_lc.lte(255)
    cond = ee.Image.cat(
    EvergreenNeedleleafForests.eq(1).rename('EvergreenNeedleleafForests5'),
    EvergreenBroadleafForests.eq(1).rename('EvergreenBroadleafForests5'),
    DeciduousNeedleleafForests.eq(1).rename('DeciduousNeedleleafForests5'),
    DeciduousBroadleafForests.eq(1).rename('DeciduousBroadleafForests5'),
    MixedForests.eq(1).rename('MixedForests5'),
    ClosedShrublands.eq(1).rename('ClosedShrublands5'),
    OpenShrublands.eq(1).rename('OpenShrublands5'),
    WoodySavannas.eq(1).rename('WoodySavannas5'),
    Savannas.eq(1).rename('Savannas5'),
    Grasslands.eq(1).rename('Grasslands5'),
    PermanentWetlands.eq(1).rename('PermanentWetlands5'),
    Croplands.eq(1).rename('Croplands5'),
    UrbanBuiltupLands.eq(1).rename('UrbanBuiltupLands5'),
    CroplandNaturalVegetationMosaics.eq(1).rename('CroplandNaturalVegetationMosaics5'),
    PermanentSnowandIce.eq(1).rename('PermanentSnowandIce5'),
    Barren.eq(1).rename('Barren5'),
    WaterBodies.eq(1).rename('WaterBodies5'),
    Unclassified.eq(1).rename('Unclassified5'),

    Land.eq(1).rename('Land5'))
    for i in range (0,yrange):
        try:
            y1 = ymin + i*dx
            y2 = ymin + (i+1)*dx
            grid = generateGrid(xmin, xmax, dx, y1, y2)
            npsRes = cond.reduceRegions(grid, ee.Reducer.sum(), 500)#500不是分辨率，改动对结果影响小
            npsRes = npsRes.map(func_feh)
            name='GLC_FCS30_'+'China_Fishnet_'+str(i)+'.csv'
            out_dir = 'G:\\LUCC_China\\MCD12Q1_LC\\CN_Fishnet_'+str(year)
            out_ndvi_stats = os.path.join(out_dir, name)
            if not os.path.exists(out_dir):
                os.makedirs(out_dir)
            # Allowed output formats: csv, shp, json, kml, kmz
            # Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM, resolution=500
            #SCALE represent the resolution of the image that can be used in .multiply(ee.Image.pixelArea())
            url = npsRes.getDownloadURL(filetype="csv", selectors=['TARGET_FID', 'EvergreenNeedleleafForests5',
                                                                    'EvergreenBroadleafForests5',
                                                                    'DeciduousNeedleleafForests5',
                                                                    'DeciduousBroadleafForests5',
                                                                    'MixedForests5',
                                                                    'ClosedShrublands5',
                                                                    'OpenShrublands5',
                                                                    'WoodySavannas5',
                                                                    'Savannas5',
                                                                    'Grasslands5',
                                                                    'PermanentWetlands5',
                                                                    'Croplands5',
                                                                    'UrbanBuiltupLands5',
                                                                    'CroplandNaturalVegetationMosaics5',
                                                                    'PermanentSnowandIce5',
                                                                    'Barren5',
                                                                    'WaterBodies5',
                                                                    'Unclassified5',
                                                                    'Land5'], filename=name)
            print(url)
            print(out_ndvi_stats)
            get_request(url)
        except Exception as e:
            print(e)       

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/3478a46478c25bf590905e7f3d36b55b-d1cdb76dcf50d54e6eec8c71c475fa45:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_9705.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/f8fc9af1d60d4551d57a0f17a9912b95-f5efccc5c4337dfce36f867a071b2059:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_9706.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/9215aaaa5825a019729d567b61649749-887327becfe8fce8395aed4d2ce60e9a:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_9707.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/8e326dc504fdc8c78b05df37ac10d2ed-376756d04752c29910cd16beb89f5b59:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_9708.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/5c

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/e63152a3f16af0fe81a8117af0e5991e-635ddbf12030a07ce6dfb256ff9b4f04:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_9741.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/8796126f163ea2b5504eaca0290a030d-995dec20b087e5fbff8fc7893cf0077b:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_9742.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/34e6eef89b85524db606b3ae3be7e8f7-0b8df2c6451bcb4f11d34d7d5a15ecf8:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_9743.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/1010554adaea8f992a23b640c74654a0-ea38b25c1a00afbf2c4ed1845643021f:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_9744.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/da

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/c2ebede1e7891987b32da42c109f3aa7-6c8e531c5d9db5b9c538e84f5ff0acf8:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_9778.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/8fa700c78cf87584c153681e81fa72ae-81b6d9466f514d6f10e9bd63ffd116ac:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_9779.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/5705dfaaf0e06cf055c37e48b7dd64ff-053cf3483a54ed74d8dc9a4e99247741:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_9780.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/dde264469f243a58eb4f1b3c1ee732a7-69d255f1094011b33ce0fcf20339c0ed:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_9781.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/be

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/1784886d47bf6178c071dedd46563934-e18f7575169d1d7c8d9f8aa54faec355:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_9814.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/af41cb23d19585162bcf3407a83ffc9a-4641ac2c6bd818c03721c2efb1d25e6d:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_9815.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/1779afb5a96f54e53af762c103206e08-cd0c3ea48bb0988767829dcfbe137644:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_9816.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/126e9ba7f0f7b9833e7568a49e9e5a05-2448385c80e444eb4e1af24bb290d57e:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_9817.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/f7

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/4fab3d4b046e49df86d97d6e438e4093-d52244ce8ba556eb9549c13c0b222a60:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_9850.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/df898c5f2e96e79d2afef3428f1b9116-a04dd91a27e0248ab41bec4d768d28c4:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_9851.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/8622a84e0a48fb00f0c3f65da1fedbc2-c0ec95e20f5dc54000a825460a8fca30:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_9852.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/9a1eae1eded0a65ac870e1a0c5169c55-b46e6b9524e9b257d668cd9944e5ea00:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_9853.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/b6

...

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/8f9f38ae79adf4c411704dad84274835-0f483f675a13ddaef0bdbb1a6993ff27:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_10211.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/c5bb22788829227507cd29f34c17de65-56ee472d0f52a55b2d441cca9102ac6b:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_10212.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/def4487eb660e1bf0e94e88ded6d2ed6-958c4373e0c66e0b33cd660fb810698e:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_10213.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/cc101c9ede0e8760933dbddd0d522c55-17bf710fe02d813f9e2844e39ba68f9b:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_10214.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/table

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/850faab20eb19dc74b1aab4217348dfe-d4398fafb209714d40be40fae3810134:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_10247.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/e32dd094967975e8bec9f35f3d9abe02-32952ea01c5358e01eb20c4d55262193:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_10248.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/583fbf40b98c8ab847dcd6212700ae0f-fe8ac10bf1c78986c2253ef17cfe22d3:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_10249.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/e95f4a3d10cde895879b2e4f299c71c7-ed15c400139f92e4fb827f053cdf9767:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_10250.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/table

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/86f290bec5f6e9aafa68a8e9ff87ace7-635abb1c0ee10334112c6161dd058ad3:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_10283.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/22f42bf4e30b06669263aa775bdcebe1-f36a87c231e596c666c4a075bac7ed4f:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_10284.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/7397319dbcc1eb00c4637746cc232974-c2232b7cf87c8753673db8bf7a7d1743:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_10285.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/43a16866d4b8c3a6fa5c63a861c77b10-847cad570fb5e49283e14e9481727f62:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_10286.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/table

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/d631d7134805884f0498dea2ced372b9-8e34990e13035d933100c21e6a3da6e4:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_10319.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/5ac452fea0e3672aa65f5743786f67a8-7563b3f2a87a0a78b62f7db6e33154f8:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_10320.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/718eba29ba123e9408fcf70c3afcef9f-84cc365fb934f1bfa539e26ed1d847cc:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_10321.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/00190165ff947e3e26ccd718abd1c2bf-fec4c70ecbbc401719551bc56921d3bb:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_10322.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/table

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/ad0f597f48f43831b8f9b6cb1347241d-ed5114dbf3c15272377c26aaa1035f57:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_10355.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/0c8371a6737d326f4e9d0a9402c22f43-312f99e1ddb598362cf33287cb309711:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_10356.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/7e4619ba854cfe14504608a884777ada-5012b3cbadb397fed41b64d134e0caf1:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_10357.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/9c1f63d259315d83d589681051096107-4acd11e62d904a1ea03892b28ecec239:getFeatures
G:\LUCC_China\MCD12Q1_LC\CN_Fishnet_2020\GLC_FCS30_China_Fishnet_10358.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/table